# Custom strategy end-to-end demo

This notebook shows how to implement a simple trading strategy, run a vectorized backtest using the project's `Backtester`, inspect results, export detected trades, and create interactive visualizations. It's intended to be runnable inside this repository's virtual environment.

In [ ]:
# Environment & imports
import sys
from pathlib import Path
# ensure project root is on sys.path so `from src...` works when running the notebook from the repo folder
ROOT = Path('..').resolve()  # notebook is under notebooks/ -> project root is ..
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# optional interactive plotting (only used later)
try:
    import plotly.graph_objects as go
    import plotly.io as pio
    PLOTLY_AVAILABLE = True
    pio.renderers.default = 'notebook'
except Exception:
    PLOTLY_AVAILABLE = False

# project backtest utilities
from src.backtest.data import download_sp500
from src.backtest.engine import Backtester

# reproducible seed for any random parts (not used heavily here)
np.random.seed(42)

## Load & inspect market data
We use the built-in `download_sp500` helper to fetch S&P 500 OHLCV data via yfinance.
We'll pick a multi-year slice for the demo.

In [ ]:
start = '2018-01-01'
end = '2020-12-31'
print(f'Downloading S&P 500 data from {start} to {end} (this may hit the network)')
df = download_sp500(start=start, end=end)
print('columns:', df.columns.tolist())
df_head = df.head()
df_tail = df.tail()
df_head

## Preprocess & feature engineering
Create a close series (prefer Adjusted Close), simple moving averages, returns, and a simple volatility measure.

In [ ]:
# pick close/adj close robustly
if 'Adj Close' in df.columns:
    close = df['Adj Close'].copy()
elif 'Close' in df.columns:
    close = df['Close'].copy()
else:
    # fallback to first numeric column
    numeric_cols = df.select_dtypes('number').columns
    close = df[numeric_cols[0]].copy()

data = pd.DataFrame({'close': close})
data['ret'] = data['close'].pct_change()
data['sma20'] = data['close'].rolling(20).mean()
data['sma50'] = data['close'].rolling(50).mean()
data['vol20'] = data['ret'].rolling(20).std()
data.dropna(inplace=True)
data.head()

## Define a custom strategy: BreakoutStrategy
This simple strategy goes long when the close breaks above the rolling max over a lookback window (breakout). Signals are shifted by 1 to represent taking the position at the next bar.

In [ ]:
class BreakoutStrategy:
    def __init__(self, lookback: int = 20):
        if lookback < 1:
            raise ValueError('lookback must be >= 1')
        self.lookback = lookback

    def generate_signals(self, prices: pd.Series) -> pd.Series:
        # rolling max excluding the current bar (shift the rolling window by 1)
        rolling_max = prices.rolling(self.lookback).max().shift(1)
        signal = (prices > rolling_max).astype(int)
        # shift again so position is applied on next bar (consistent with Backtester convention)
        return signal.shift(1).fillna(0).astype(int)

# small smoke test for the class
prices_sample = data['close'].iloc[:100]
strat = BreakoutStrategy(lookback=20)
signals_sample = strat.generate_signals(prices_sample)
print('Signals sample value counts:', signals_sample.value_counts().to_dict())

In [ ]:
# Run a backtest using the project's Backtester (vectorized, uses close-to-close returns)
bt = Backtester(data['close'])
signals = strat.generate_signals(data['close'])
res = bt.run(signals)
print(f'cumulative return: {res.cumulative_return:.2%}')
print(f'annualized return: {res.annualized_return:.2%}')
print(f'max drawdown: {res.max_drawdown:.2%}')

# simple equity plot
plt.figure(figsize=(10, 4))
res.equity.plot()
plt.title('Strategy equity (growth of 1.0)')
plt.xlabel('Date')
plt.grid(True)
plt.show()

In [ ]:
# Build trade log (detect 0->1 buys and 1->0 sells) and save CSV
pos = signals.reindex(data.index).fillna(0).astype(int)
trd = pos.diff().fillna(0)
buys = trd[trd == 1].index
sells = trd[trd == -1].index

def get_exec_price(idx):
    # try Open if available, otherwise close
    if 'Open' in df.columns:
        return df['Open'].reindex(idx).to_numpy()
    return df['Adj Close' if 'Adj Close' in df.columns else 'Close'].reindex(idx).to_numpy()

trade_rows = []
for t in buys:
    p = float(get_exec_price([t])[0])
    trade_rows.append({'timestamp': t, 'side': 'buy', 'price': p, 'size': 1.0})
for t in sells:
    p = float(get_exec_price([t])[0])
    trade_rows.append({'timestamp': t, 'side': 'sell', 'price': p, 'size': 1.0})

trades_df = pd.DataFrame(trade_rows).sort_values('timestamp')
out_dir = Path('../reports_output').resolve()
out_dir.mkdir(parents=True, exist_ok=True)
csv_path = out_dir / 'notebook_trades.csv'
trades_df.to_csv(csv_path, index=False)
print('Saved trades to:', csv_path)
trades_df.head()

In [ ]:
# Interactive plot: use plotly if available, otherwise matplotlib fallback
if PLOTLY_AVAILABLE:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close'))
    # add trade markers
    if len(buys) > 0:
        fig.add_trace(go.Scatter(x=buys, y=get_exec_price(buys), mode='markers', marker=dict(symbol='triangle-up', color='green', size=10), name='Buys', hovertemplate='%{x}<br>Buy: %{y:.2f}'))
    if len(sells) > 0:
        fig.add_trace(go.Scatter(x=sells, y=get_exec_price(sells), mode='markers', marker=dict(symbol='triangle-down', color='red', size=10), name='Sells', hovertemplate='%{x}<br>Sell: %{y:.2f}'))
    fig.update_layout(title='Price with trades', xaxis_title='Date', yaxis_title='Price')
    fig.show()
else:
    plt.figure(figsize=(12, 5))
    plt.plot(data.index, data['close'], label='Close')
    if len(buys) > 0:
        plt.scatter(buys, get_exec_price(buys), marker='^', color='green', s=80, label='Buys')
    if len(sells) > 0:
        plt.scatter(sells, get_exec_price(sells), marker='v', color='red', s=80, label='Sells')
    plt.legend()
    plt.title('Price with trades (static)')
    plt.show()

## Next steps
- Tune the strategy hyperparameters (lookback, threshold).
- Add transaction costs and slippage into the backtester.
- Implement parameter grid search and walk-forward validation.
- Replace the simple Backtester with a sizing-aware engine if you want variable position sizes.
- Use interactive Plotly output inside Jupyter for cleaner exploration.